In [1]:
import pandas as pd
import sweetviz as sv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [2]:
df_clean_cat = pd.read_csv('./csv/clean_order_v2.csv')
df_clean_cat

,customer_id,order_product_id,browser,platform,customer_firstname,customer_lastname,customer_email,customer_gender,Majority_code,order_price_paid,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,0,121,2,0,Julie,Staples,juliestaples40@gmail.com,2,77,4500,...,0,0,0,0,0,0,0,0,0,0
1,0,121,2,0,Alana,Locandro,alana.lannie@gmail.com,1,13,4500,...,0,0,0,0,0,0,0,0,0,0
2,0,121,7,6,Harriet,Peace,Harrietpeace@icloud.com,1,77,4500,...,0,0,0,0,0,0,0,0,0,0
3,221,121,8,0,Kain,Ingham,Kaino_01@hotmail.com,0,13,4500,...,0,0,0,0,0,0,0,0,0,0
4,0,121,2,4,Sandra,Nicholl,sandinicholl@hotmail.co.uk,2,77,15750,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,0,202,2,4,Ali,Mohammad,fourtwenty.aggregation@gmail.com,0,38,3580,...,0,0,0,0,0,1,0,0,0,0
5513,0,202,2,4,Kirill,Kiryushin,kirill.kiryushin.o@gmail.com,0,218,3980,...,0,0,0,0,0,1,0,0,0,0
5514,0,202,2,4,Paitoon,Uthaisang,puthaisang@gmail.com,0,218,7020,...,0,0,0,0,0,1,0,0,0,0
5515,0,202,2,0,Alex,Schetinin,shatz.g@gmail.com,0,103,3980,...,0,0,0,0,0,1,0,0,0,0


In [4]:
# fix customer_id is 0 or 23
df_clean_cat['new_id'] = df_clean_cat['customer_id']
df_clean_cat['new_firstname'] = df_clean_cat['customer_firstname'].str.lower()
df_clean_cat['new_lastname'] = df_clean_cat['customer_lastname'].str.lower()

df_clean_cat['new_email'] = df_clean_cat['customer_email'].str.lower()
email_dict = df_clean_cat['new_email'].value_counts()
del email_dict['support@daytriptour.com']

# replace customer_id [0,23]
start_val = 10000
for i in email_dict.keys():
    x = df_clean_cat.query(f'new_email == "{str(i)}"')['customer_id'].unique().tolist()
    if 0 in x: x.remove(0)
    if 23 in x: x.remove(23)
    if len(x) > 0:
        val_id = max(x)
        idx = df_clean_cat.query(f'new_email == "{str(i)}"').index
        for j in idx:
            if df_clean_cat.loc[j]['customer_id'] in [0,23]:
                df_clean_cat.loc[j,['new_id']] = val_id
    else:
        idx = df_clean_cat.query(f'new_email == "{str(i)}"').index
        for j in idx:
            df_clean_cat.loc[j,['new_id']] = start_val
        start_val+=1

In [5]:
# support@daytriptour.com [0,23] has 421 items
df_clean_cat['new_id'].value_counts()

23       215
0        204
2704       9
2705       7
10000      6
        ... 
13336      1
376        1
13337      1
401        1
13339      1
Name: new_id, Length: 4595, dtype: int64

In [8]:
email_dict_daytriptour = df_clean_cat.query('new_email == "support@daytriptour.com"')['new_lastname'].value_counts()
# start_val =13340
for i in email_dict_daytriptour.keys():
    x = df_clean_cat.query(f'new_lastname == "{str(i)}" & new_email == "support@daytriptour.com"')['new_firstname'].unique().tolist()
    if len(x) == 1:
        idx = df_clean_cat.query(f'new_lastname == "{str(i)}" & new_email == "support@daytriptour.com"').index
        for j in idx:
            df_clean_cat.loc[j,['new_id']] = start_val
        start_val+=1
    else:
        for k in x:
            idx = df_clean_cat.query(f'new_lastname == "{str(i)}" & new_firstname == "{str(k)}" & new_email == "support@daytriptour.com"').index
            for j in idx:
                df_clean_cat.loc[j,['new_id']] = start_val
            start_val+=1
print(start_val)

13739


In [9]:
df_clean_cat['new_id'].value_counts()

2704     9
2705     7
10000    6
10001    5
888      5
        ..
2428     1
13607    1
13608    1
13609    1
13339    1
Name: new_id, Length: 4992, dtype: int64

In [10]:
df_clean_cat

,customer_id,order_product_id,browser,platform,customer_firstname,customer_lastname,customer_email,customer_gender,Majority_code,order_price_paid,...,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,new_id,new_firstname,new_lastname,new_email
0,0,121,2,0,Julie,Staples,juliestaples40@gmail.com,2,77,4500,...,0,0,0,0,0,0,10531,julie,staples,juliestaples40@gmail.com
1,0,121,2,0,Alana,Locandro,alana.lannie@gmail.com,1,13,4500,...,0,0,0,0,0,0,12774,alana,locandro,alana.lannie@gmail.com
2,0,121,7,6,Harriet,Peace,Harrietpeace@icloud.com,1,77,4500,...,0,0,0,0,0,0,12285,harriet,peace,harrietpeace@icloud.com
3,221,121,8,0,Kain,Ingham,Kaino_01@hotmail.com,0,13,4500,...,0,0,0,0,0,0,221,kain,ingham,kaino_01@hotmail.com
4,0,121,2,4,Sandra,Nicholl,sandinicholl@hotmail.co.uk,2,77,15750,...,0,0,0,0,0,0,12286,sandra,nicholl,sandinicholl@hotmail.co.uk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,0,202,2,4,Ali,Mohammad,fourtwenty.aggregation@gmail.com,0,38,3580,...,0,1,0,0,0,0,11192,ali,mohammad,fourtwenty.aggregation@gmail.com
5513,0,202,2,4,Kirill,Kiryushin,kirill.kiryushin.o@gmail.com,0,218,3980,...,0,1,0,0,0,0,11193,kirill,kiryushin,kirill.kiryushin.o@gmail.com
5514,0,202,2,4,Paitoon,Uthaisang,puthaisang@gmail.com,0,218,7020,...,0,1,0,0,0,0,11194,paitoon,uthaisang,puthaisang@gmail.com
5515,0,202,2,0,Alex,Schetinin,shatz.g@gmail.com,0,103,3980,...,0,1,0,0,0,0,11195,alex,schetinin,shatz.g@gmail.com


In [14]:
df_clean_cat_v3 = df_clean_cat[[
    'customer_id', 'new_id', 'order_product_id', 'browser', 'platform',
    'customer_firstname', 'customer_lastname', 'customer_email',
    'customer_gender', 'Majority_code', 'order_price_paid',
    'order_payment_by', 'booked_days', 'departure_year', 'departure_month',
    'departure_day', 'departure_DayofYear', 'departure_DayofWeek',
    'sum_kids', 'sum_adults', 'private', 'group', 'family', 'cat_1',
    'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9',
    'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16',
    'cat_17', 'cat_18']]

In [20]:
df_clean_cat_v3[df_clean_cat_v3['new_id'].duplicated()].sort_values('new_id')

,customer_id,new_id,order_product_id,browser,platform,customer_firstname,customer_lastname,customer_email,customer_gender,Majority_code,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
5116,0,8,181,2,4,Kantima,Martpadup,dolavic@hotmail.com,1,218,...,0,0,0,0,0,0,0,0,1,0
2151,8,8,52,7,6,Alicia,Ling,dolavic@hotmail.com,0,198,...,1,0,0,0,0,0,0,0,0,0
3898,0,8,156,7,6,Michael,Ellis,dolavic@hotmail.com,0,38,...,0,0,0,0,1,0,0,0,0,0
4290,0,8,182,7,6,Anand,Kunar,dolavic@hotmail.com,0,105,...,0,0,0,0,0,0,0,0,1,0
1248,200,200,46,7,6,Petagaye,Whyte,wise1988@yahoo.com,1,233,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5103,23,13367,181,7,6,Peenit,Changngan,support@daytriptour.com,2,218,...,0,0,0,0,0,0,0,0,1,0
4809,0,13368,181,7,6,Natjira,Kongpetchs,support@daytriptour.com,1,218,...,0,0,0,0,0,0,0,0,1,0
5150,0,13369,181,2,4,Pornthip,Sitthisanguanthai,support@daytriptour.com,1,218,...,0,0,0,0,0,0,0,0,1,0
4702,23,13372,181,7,6,Pantouch,Vattanayagorn,support@daytriptour.com,0,218,...,0,0,0,0,0,0,0,0,1,0


In [22]:
df_clean_cat_v3[df_clean_cat_v3['new_id'].duplicated()]['new_id'].unique()

array([  420,   616, 10176, 10230,   441, 10071, 10090, 10018, 10062,
        2541,   222, 10000, 10213,   802, 10194,   939, 10074, 10131,
        1278, 10012,  1857,  2792,   200, 10218, 10228, 10236, 10036,
       10129, 10243, 10020, 10190,   998, 10053,  1317, 10139,  1593,
       10024,  1748, 10198,   205, 10150, 10239, 10206, 10179, 10220,
       10109,   294, 10202, 10214, 10221, 10188, 10183,  1068, 10032,
       10002, 10008,   606,   937,  1313,  2705, 10011, 10162,   529,
       10083, 10017,   258,  1758, 10125, 10037, 10175, 10113, 10038,
         336,  1098, 10041, 10240,   478, 10078,  1833,   221,   244,
       10169, 10107,   787, 10048, 10003, 10034, 10039, 10210, 10088,
       10219,  1157, 10013, 10197, 10001,  1658, 10096, 10122,  1983,
       10211, 10097,   608,   613,   678,   962, 10114, 10028,  1637,
       10155, 10004, 10061, 10163, 10165,   228, 10224,     8, 10231,
         668, 10045,  1052, 13366, 10027,   285, 10105, 10022, 10143,
       10144, 10102,

In [23]:
df_clean_cat_v3.to_csv('./csv/df_clean_cat_v3.csv', index=False)